In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from dnn_tau import Dnn_tau
from data_extractor import Data_extractor_v4, output_vars_v4
import os
import fnmatch
from utils import normalize, bucketize, split_dataset, flatten_2D_list
from copy import deepcopy
import pickle

In [2]:
path = "/data/hnl/prompt_tau/anatuple/nanoV10/TEST9/"
features = deepcopy(output_vars_v4)
features.extend(['signal_label', 'channel', 'event_type', 'mass_hyp'])
channels = os.listdir(path)
relative_path = "/anatuple/"

In [5]:
values = []
flat_features = flatten_2D_list(features)
for i in range(len(flat_features)):
    values.append([])
data = dict(zip(flat_features, values))

for channel in channels:
    extractor = Data_extractor_v4(channel)
    data = extractor(path+channel+relative_path, data=data)

In [6]:
data_dict = data

In [10]:
data = pd.DataFrame(data_dict)
data = data.rename(columns={"genWeight": "weightOriginal"})
weightNorm = deepcopy(data['weightOriginal'])
data['weightNorm'] = weightNorm

In [14]:
N = len(data['event'])
data_norm = normalize(pd.DataFrame(data), 'mass_hyp', N, weight_name='weightNorm')
data_norm = normalize(data_norm, 'signal_label', N, weight_name='weightNorm')
data_norm = normalize(data_norm, 'channel', N/5, weight_name='weightNorm')
data_processed, channel_indices = bucketize(data_norm, 'channel')
print(list(data_processed.keys()))
print(channel_indices)
print(N)
print(sum(data_processed['weightNorm']))

['event', 'weightOriginal', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'mass_12', 'mass_13', 'mass_23', 'mass_123', 'Mt_tot', 'HNL_CM_angle_with_MET_1', 'HNL_CM_angle_w

In [15]:
with open("extracted_data/TEST9_global_v4_all_normalized_channel_indices", 'wb') as file:
    pickle.dump(channel_indices, file)

In [16]:
data_processed.to_pickle("extracted_data/TEST9_global_v4_og_and_norm_weights")

In [25]:
selection = ['charge_1', 'charge_2', 'charge_3', 'pt_1',
       'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2',
       'mass_3', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET',
       'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)',
       'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)',
       'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)',
       'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)',
       'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23',
       'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12',
       'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)',
       'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET',
       'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)',
       'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)',
       'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'mass_12', 'mass_13',
       'mass_23', 'mass_123', 'Mt_tot', 'HNL_CM_angle_with_MET_1',
       'HNL_CM_angle_with_MET_2', 'W_CM_angle_to_plane_1',
       'W_CM_angle_to_plane_2', 'W_CM_angle_to_plane_with_MET_1',
       'W_CM_angle_to_plane_with_MET_2', 'HNL_CM_mass_1', 'HNL_CM_mass_2',
       'HNL_CM_mass_with_MET_1', 'HNL_CM_mass_with_MET_2', 'W_CM_angle_12',
       'W_CM_angle_13', 'W_CM_angle_23', 'W_CM_angle_1MET', 'W_CM_angle_2MET',
       'W_CM_angle_3MET', 'mass_hyp', 'signal_label']

In [17]:
train, val, test, meas = split_dataset(data_processed)
pd.to_pickle(train, "extracted_data/TEST9_global_v4_og_and_norm_weights_train")
pd.to_pickle(val, "extracted_data/TEST9_global_v4_og_and_norm_weights_val")
pd.to_pickle(test, "extracted_data/TEST9_global_v4_og_and_norm_weights_test")
pd.to_pickle(meas, "extracted_data/TEST9_global_v4_og_and_norm_weights_meas")

Total number of events :  1610653
Train set : 37.52 %
Validation set : 12.51 %
Test set : 25.00 %
Measurement set : 24.98 %
